In [1]:
%%capture
# Import libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils.validation import check_is_fitted

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src import config, data_utils, preprocessing

In [2]:
app_train, app_test, columns_description = data_utils.get_datasets()

In [3]:
X_train, y_train, X_test, y_test = data_utils.get_feature_target(app_train, app_test)
X_train, X_val, y_train, y_val = data_utils.get_train_val_sets(X_train, y_train)
train_data, val_data, test_data = preprocessing.preprocess_data(X_train, X_val, X_test)

In [6]:
import mlflow
from sklearn.metrics import accuracy_score


C= 0.0001
max_iter= 1000
random_state= 42

with mlflow.start_run(run_name= "Baseline"):
    log_reg = LogisticRegression(C=C,
                                max_iter=max_iter,
                                random_state=random_state)

    log_reg.fit(train_data, y_train)

    # log
    mlflow.log_param("C", C)
    mlflow.log_param("max_iter", 1000)
    mlflow.log_param("random_state", random_state)


    # Calcular y registrar la métrica de precisión
    train_preds = log_reg.predict(train_data)
    train_accuracy = accuracy_score(y_train, train_preds)
    mlflow.log_metric("train_accuracy", train_accuracy)

    # Registrar el modelo
    mlflow.sklearn.log_model(log_reg, "logistic_regression")

c:\Users\crist\miniconda3\envs\gpu_base\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


### Simple LR

In [25]:
import mlflow
from sklearn.metrics import accuracy_score, roc_auc_score

C = 1
max_iter = 1000
random_state = 42

with mlflow.start_run(run_name="LogisticRegression"):
    log_reg = LogisticRegression(C=C,
                                  max_iter=max_iter,
                                  random_state=random_state)

    log_reg.fit(train_data, y_train)

    # Log parameters
    mlflow.log_param("Model", "LogisticRegression")
    mlflow.log_param("C", C)
    mlflow.log_param("max_iter", max_iter)
    mlflow.log_param("random_state", random_state)

    # Calcular y registrar la métrica de precisión
    train_preds = log_reg.predict(train_data)
    train_accuracy = accuracy_score(y_train, train_preds)
    mlflow.log_metric("train_accuracy", train_accuracy)

    # Calcular y registrar el ROC AUC para el conjunto de entrenamiento
    train_probs = log_reg.predict_proba(train_data)[:, 1]
    train_roc_auc = roc_auc_score(y_train, train_probs)
    mlflow.log_metric("train_roc_auc", train_roc_auc)

    # Registrar el modelo
    mlflow.sklearn.log_model(log_reg, "logistic_regression")

## Medium - RFC

In [31]:
import mlflow
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

def train_and_log_model(train_data, y_train, val_data, y_val, n_estimators, max_depth, random_state, run_name="Baseline"):
    with mlflow.start_run(run_name=run_name):
        rf_clf = RandomForestClassifier(n_estimators=n_estimators,
                                        max_depth=max_depth,
                                        random_state=random_state)

        rf_clf.fit(train_data, y_train)

        mlflow.log_param("Model", run_name)


        # Log parameters
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("random_state", random_state)

        # Calcular y registrar la métrica de precisión
        train_preds = rf_clf.predict(train_data)
        train_accuracy = accuracy_score(y_train, train_preds)
        mlflow.log_metric("train_accuracy", train_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de entrenamiento
        train_probs = rf_clf.predict_proba(train_data)[:, 1]
        train_roc_auc = roc_auc_score(y_train, train_probs)
        mlflow.log_metric("train_roc_auc", train_roc_auc)

        # Registrar el modelo
        mlflow.sklearn.log_model(rf_clf, "random_forest_classifier")

        # Calcular y registrar el ROC AUC para el conjunto de validación
        val_probs = rf_clf.predict_proba(val_data)[:, 1]
        val_roc_auc = roc_auc_score(y_val, val_probs)
        mlflow.log_metric("val_roc_auc", val_roc_auc)

        print(f"ROC AUC en el conjunto de validación: {val_roc_auc:.4f}")




In [32]:
# Ejemplo de uso de la función
train_and_log_model(train_data, y_train, val_data, y_val, n_estimators=60, max_depth=5, random_state=42, run_name="RandomForestClassifier")

ROC AUC en el conjunto de validación: 0.7151


In [28]:
list_max_depth_case = [1,2,3,4,5,6,7]

for i in list_max_depth_case:
    train_and_log_model(train_data, y_train, val_data, y_val, n_estimators=100, max_depth= i, random_state=42, run_name="RandomForestClassifier")

ROC AUC en el conjunto de validación: 0.7047
ROC AUC en el conjunto de validación: 0.7122
ROC AUC en el conjunto de validación: 0.7117
ROC AUC en el conjunto de validación: 0.7145
ROC AUC en el conjunto de validación: 0.7171
ROC AUC en el conjunto de validación: 0.7218
ROC AUC en el conjunto de validación: 0.7245


## Random search - RFC

In [33]:
import mlflow
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

def train_and_log_model(train_data, y_train, val_data, y_val, n_iter, cv, random_state,
                        n_estimators_range, max_depth_range, min_samples_split_values,
                        min_samples_leaf_values, bootstrap_values, run_name="RandomizedSearchCV"):
    
    param_grid = {
        "n_estimators": n_estimators_range,
        "max_depth": max_depth_range,
        "min_samples_split": min_samples_split_values,
        "min_samples_leaf": min_samples_leaf_values,
        "bootstrap": bootstrap_values,
    }

    rf_random = RandomizedSearchCV(
        estimator=RandomForestClassifier(),
        param_distributions=param_grid,
        n_iter=n_iter,
        cv=cv,
        verbose=2,
        random_state=random_state,
    )

    with mlflow.start_run(run_name=run_name):
        rf_random.fit(train_data, y_train)

        best_params = rf_random.best_params_
        best_rf_clf = rf_random.best_estimator_

        # Log parameters
        mlflow.log_param("Model", run_name)
        for param, value in best_params.items():
            mlflow.log_param(param, value)

        # Calcular y registrar la métrica de precisión
        train_preds = best_rf_clf.predict(train_data)
        train_accuracy = accuracy_score(y_train, train_preds)
        mlflow.log_metric("train_accuracy", train_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de entrenamiento
        train_probs = best_rf_clf.predict_proba(train_data)[:, 1]
        train_roc_auc = roc_auc_score(y_train, train_probs)
        mlflow.log_metric("train_roc_auc", train_roc_auc)

        # Registrar el modelo
        mlflow.sklearn.log_model(best_rf_clf, "random_forest_classifier")

        # Calcular y registrar el ROC AUC para el conjunto de validación
        val_probs = best_rf_clf.predict_proba(val_data)[:, 1]
        val_roc_auc = roc_auc_score(y_val, val_probs)
        mlflow.log_metric("val_roc_auc", val_roc_auc)

        print(f"ROC AUC en el conjunto de validación: {val_roc_auc:.4f}")


In [34]:
# Ejemplo de uso de la función
n_estimators_range = [int(x) for x in np.linspace(start=200, stop=2000, num=50)]
max_depth_range = [int(x) for x in np.linspace(10, 50, num=5)]
min_samples_split_values = [2, 5]
min_samples_leaf_values = [1, 2]
bootstrap_values = [True]

train_and_log_model(train_data, y_train, val_data, y_val, n_iter=2, cv=2, random_state=42,
                    n_estimators_range=n_estimators_range, max_depth_range=max_depth_range,
                    min_samples_split_values=min_samples_split_values, min_samples_leaf_values=min_samples_leaf_values,
                    bootstrap_values=bootstrap_values)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=273; total time= 1.3min
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=273; total time= 1.3min
[CV] END bootstrap=True, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=1485; total time=12.2min
[CV] END bootstrap=True, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=1485; total time=11.2min
ROC AUC en el conjunto de validación: 0.7324


## Grid search - RFC

In [35]:
import mlflow
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

def train_and_log_model(train_data, y_train, val_data, y_val, cv, random_state,
                        n_estimators_range, max_depth_range, min_samples_split_values,
                        min_samples_leaf_values, bootstrap_values, run_name="GridSearchCV"):
    
    param_grid = {
        "n_estimators": n_estimators_range,
        "max_depth": max_depth_range,
        "min_samples_split": min_samples_split_values,
        "min_samples_leaf": min_samples_leaf_values,
        "bootstrap": bootstrap_values,
    }

    rf_grid = GridSearchCV(
        estimator=RandomForestClassifier(),
        param_grid=param_grid,
        cv=cv,
        verbose=2,
        n_jobs=-1,
    )

    with mlflow.start_run(run_name=run_name):
        rf_grid.fit(train_data, y_train)

        best_params = rf_grid.best_params_
        best_rf_clf = rf_grid.best_estimator_

        # Log parameters
        for param, value in best_params.items():
            mlflow.log_param(param, value)

        # Calcular y registrar la métrica de precisión en el conjunto de entrenamiento
        train_preds = best_rf_clf.predict(train_data)
        train_accuracy = accuracy_score(y_train, train_preds)
        mlflow.log_metric("train_accuracy", train_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de entrenamiento
        train_probs = best_rf_clf.predict_proba(train_data)[:, 1]
        train_roc_auc = roc_auc_score(y_train, train_probs)
        mlflow.log_metric("train_roc_auc", train_roc_auc)

        # Calcular y registrar la métrica de precisión en el conjunto de validación
        val_preds = best_rf_clf.predict(val_data)
        val_accuracy = accuracy_score(y_val, val_preds)
        mlflow.log_metric("val_accuracy", val_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de validación
        val_probs = best_rf_clf.predict_proba(val_data)[:, 1]
        val_roc_auc = roc_auc_score(y_val, val_probs)
        mlflow.log_metric("val_roc_auc", val_roc_auc)

        # Registrar el modelo
        mlflow.sklearn.log_model(best_rf_clf, "random_forest_classifier")

        print(f"ROC AUC en el conjunto de validación: {val_roc_auc:.4f}")

# Ejemplo de uso de la función
n_estimators_range = [int(x) for x in np.linspace(start=200, stop=2000, num=50)]
max_depth_range = [int(x) for x in np.linspace(10, 50, num=5)]
min_samples_split_values = [2, 5]
min_samples_leaf_values = [1, 2]
bootstrap_values = [True]

train_and_log_model(train_data, y_train, val_data, y_val, cv=2, random_state=42,
                    n_estimators_range=n_estimators_range, max_depth_range=max_depth_range,
                    min_samples_split_values=min_samples_split_values, min_samples_leaf_values=min_samples_leaf_values,
                    bootstrap_values=bootstrap_values)


Fitting 2 folds for each of 1000 candidates, totalling 2000 fits


: 

## LightGBM

In [27]:
import mlflow
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score

def train_and_log_lightgbm(train_data, y_train, val_data, y_val, params, run_name="LightGBM"):
    
    with mlflow.start_run(run_name=run_name):

        # Crear el conjunto de datos de LightGBM
        lgb_train_data = lgb.Dataset(train_data, label=y_train)
        lgb_val_data = lgb.Dataset(val_data, label=y_val, reference=lgb_train_data)

        mlflow.log_param("Model", run_name)

        # Entrenar el modelo LightGBM
        lgb_model = lgb.train(params, lgb_train_data, valid_sets=lgb_val_data)

        # Log parameters
        for param, value in params.items():
            mlflow.log_param(param, value)

        # Calcular y registrar la métrica de precisión en el conjunto de entrenamiento
        train_preds = np.round(lgb_model.predict(train_data))
        train_accuracy = accuracy_score(y_train, train_preds)
        mlflow.log_metric("train_accuracy", train_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de entrenamiento
        train_probs = lgb_model.predict(train_data)
        train_roc_auc = roc_auc_score(y_train, train_probs)
        mlflow.log_metric("train_roc_auc", train_roc_auc)

        # Calcular y registrar la métrica de precisión en el conjunto de validación
        val_preds = np.round(lgb_model.predict(val_data))
        val_accuracy = accuracy_score(y_val, val_preds)
        mlflow.log_metric("val_accuracy", val_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de validación
        val_probs = lgb_model.predict(val_data)
        val_roc_auc = roc_auc_score(y_val, val_probs)
        mlflow.log_metric("val_roc_auc", val_roc_auc)

        # Registrar el modelo
        mlflow.lightgbm.log_model(lgb_model, "lightgbm")

        print(f"ROC AUC en el conjunto de validación: {val_roc_auc:.4f}")




In [32]:
%%capture
params = {
    "objective": "binary",
    "metric": "auc",
    "boosting_type": "gbdt",
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
}

train_and_log_lightgbm(train_data, y_train, val_data, y_val, params)

In [15]:
import mlflow
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import os
import shutil

def train_and_log_xgboost(train_data, y_train, val_data, y_val, params, run_name="XGBoost", artifact_path="artifact"):
    
    with mlflow.start_run(run_name=run_name):

        # Crear el conjunto de datos de XGBoost
        xgb_train_data = xgb.DMatrix(train_data, label=y_train)
        xgb_val_data = xgb.DMatrix(val_data, label=y_val)

        # Entrenar el modelo XGBoost
        xgb_model = xgb.train(params, xgb_train_data, evals=[(xgb_val_data, 'validation')])

        # Log parameters
        for param, value in params.items():
            mlflow.log_param(param, value)

        # Registrar el nombre del modelo
        mlflow.log_param("Model", run_name)

        # Calcular y registrar la métrica de precisión en el conjunto de entrenamiento
        train_preds = np.round(xgb_model.predict(xgb_train_data))
        train_accuracy = accuracy_score(y_train, train_preds)
        mlflow.log_metric("train_accuracy", train_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de entrenamiento
        train_probs = xgb_model.predict(xgb_train_data)
        train_roc_auc = roc_auc_score(y_train, train_probs)
        mlflow.log_metric("train_roc_auc", train_roc_auc)

        # Calcular y registrar la métrica de precisión en el conjunto de validación
        val_preds = np.round(xgb_model.predict(xgb_val_data))
        val_accuracy = accuracy_score(y_val, val_preds)
        mlflow.log_metric("val_accuracy", val_accuracy)

        # Calcular y registrar el ROC AUC para el conjunto de validación
        val_probs = xgb_model.predict(xgb_val_data)
        val_roc_auc = roc_auc_score(y_val, val_probs)
        mlflow.log_metric("val_roc_auc", val_roc_auc)

        # Registrar el modelo
        mlflow.xgboost.log_model(xgb_model, "xgboost")

        # Guardar artefacto
        os.makedirs(artifact_path, exist_ok=True)
        
        # Crear y guardar la matriz de confusión como imagen
        cm = confusion_matrix(y_val, val_preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
        plt.ylabel("True label")
        plt.xlabel("Predicted label")
        plt.title("Matriz de confusión")
        confusion_matrix_path = os.path.join(artifact_path, "confusion_matrix.png")
        plt.savefig(confusion_matrix_path)
        plt.close()

        mlflow.log_artifacts(artifact_path)

        # Eliminar el directorio del artefacto local después de registrar
        shutil.rmtree(artifact_path)

        print(f"ROC AUC en el conjunto de validación: {val_roc_auc:.4f}")



In [33]:
%%capture
# Ejemplo de uso de la función
params = {
    "objective": "binary:logistic",
    "eval_metric": "auc",
        "max_depth": 6,
    "eta": 0.3,
    "silent": 1,
}

train_and_log_xgboost(train_data, y_train, val_data, y_val, params)


## Imagenes

In [1]:
%%capture
import os
import sys


from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from src import data_utils, models, config

# # /////////////////////////////////////////////////////////////////////////////////////////////

from src import class_Experiment
from src.class_Experiment import registrar_experiment
from src import models, data_utils, config

# /////////////////////////////////////////////////////////////////////////////////////////////

DATASET_FOLDER = config.DATASET_FILENAME_IMAGE

img_height = 224
img_width = 224
channels = 3
batch_size = 32
num_classes = 25

input_shape = (img_height, img_width, channels)

# Load train and test datasets
train_ds = keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(DATASET_FOLDER, "train"),
    labels="inferred",
    label_mode="categorical",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

test_ds = keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(DATASET_FOLDER, "test"),
    labels="inferred",
    label_mode="categorical",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

class_names = train_ds.class_names
print(class_names)

assert len(class_names) == 25


# Configure data loader for performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [6]:
# /////////////////////////////////////////////////////////////////////////////////////////////
# /////////////////////////////////////////////////////////////////////////////////////////////
# /////////////////////////////////////////////////////////////////////////////////////////////
# /////////////////////////////////////////////////////////////////////////////////////////////
# Parámetros del experimento
experiment_params = {
    "experiment_name": "CV",
    "model_name": "efficientnet_B0",
    "input_shape": input_shape,
    "dropout": 0.15,
    "num_classes": len(class_names),
    "ruta_carpeta_train": r"C:\Users\crist\Documents\anyoneai\sprint_4\dataset\eu-car-dataset_subset\train",
    "ruta_carpeta_test": r"C:\Users\crist\Documents\anyoneai\sprint_4\dataset\eu-car-dataset_subset\test",
    "batch_size": 32,
    "epochs": 20,
    "data_augmentation": True,
    "additional_layers": 1,
    "neurons_per_layer": [32],
    "hyperparameters": {
        "optimizer": "adam",
        "loss": "categorical_crossentropy",
        "batch_size": 32,
        "epochs": 20,
    },
    "metrics": {
        "accuracy": "accuracy",
    },
    "augmentation_params": {
        "rotation_range": 10,
        "width_shift_range": 0.2,
        "height_shift_range": 0.1,
        "shear_range": 0.2,
        "zoom_range": 0.2,
        "horizontal_flip": True,
        "fill_mode": "nearest",
    }
}

model = models.create_efficientnet_model_compiled(
    version=0,
    input_shape=experiment_params["input_shape"],
    debug=True,
    dropout=experiment_params["dropout"],
    num_classes=experiment_params["num_classes"],
    additional_layers=experiment_params["additional_layers"],
    neurons_per_layer=experiment_params["neurons_per_layer"]

)

experiment = class_Experiment.Experiment(
    experiment_name=experiment_params["experiment_name"],
    model_name=experiment_params["model_name"],
    model=model,
    hyperparameters=experiment_params["hyperparameters"],
    metrics=experiment_params["metrics"],
    ruta_carpeta_train=experiment_params["ruta_carpeta_train"],
    ruta_carpeta_test=experiment_params["ruta_carpeta_test"],
    batch_size=experiment_params["batch_size"],
    epochs=experiment_params["epochs"],
    data_augmentation=experiment_params["data_augmentation"],
    **experiment_params["augmentation_params"]
)

class_Experiment.registrar_experiment(experiment, experiment_params["experiment_name"])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 flatten_3 (Flatten)         (None, 62720)             0         
                                                                 
 dense_6 (Dense)             (None, 32)                2007072   
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_7 (Dense)             (None, 25)                825       
                                                                 
Total params: 6,057,468
Trainable params: 2,007,897
Non-trainable params: 4,049,571
_________________________________________________________________
None
Found 1875 images belonging to 